In [ ]:
pip install --no-cache-dir llama-cpp-python==0.2.77 --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu124

In [ ]:
!pip install -q pypdf
# !pip install -q python-dotenv
!pip install torch
!pip install -q transformers
!pip -q install sentence-transformers
!pip install -q llama-index

In [ ]:
%pip install llama-index-embeddings-huggingface
%pip install llama-index-llms-llama-cpp

In [ ]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.llms.llama_cpp import LlamaCPP
from llama_index.llms.llama_cpp.llama_utils import (
    messages_to_prompt,
    completion_to_prompt,
)

In [ ]:
import torch

# from llama_index.llms import LlamaCPP
# from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt
llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    model_url='https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/resolve/main/mistral-7b-instruct-v0.2.Q4_K_M.gguf',
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=None,
    temperature=0.1,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=4096,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": -1},
    # transform inputs into Llama2 format
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
)


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving 2018 Corporate Sustainability Report English_pdf.pdf to 2018 Corporate Sustainability Report English_pdf.pdf


In [ ]:
# from llama_index import SimpleDirectoryReader
documents = SimpleDirectoryReader(
    input_files = ["2018 Corporate Sustainability Report English_pdf.pdf"]
).load_data()

In [ ]:
from llama_index.core import Document

documents = Document(text = "\n\n".join([doc.text for doc in documents]))

In [ ]:
# Putting all it together
import os
from llama_index.core.node_parser import SentenceWindowNodeParser
from llama_index.core import VectorStoreIndex, ServiceContext
from llama_index.core import Settings

def get_build_index(documents, llm, embed_model="local:BAAI/bge-small-en-v1.5", sentence_window_size=3 ,save_dir="./vector_index/index"):

  node_parser = SentenceWindowNodeParser(
      window_size = sentence_window_size,
      window_metadata_key = "window",
      original_text_metadata_key = "original_text"
  )

#   sentence_context = ServiceContext.from_defaults(
#       llm = llm,
#       embed_model= embed_model,
#       node_parser = node_parser,
#   )

  Settings.llm = llm
  Settings.embed_model = embed_model
  Settings.node_parser = node_parser

  if True: # not os.path.exists(save_dir):
        # create and load the index
        index = VectorStoreIndex.from_documents(
            [documents],
            # service_context=sentence_context,
        )
        index.storage_context.persist(persist_dir=save_dir)
#   else:
#       # load the existing index
#       index = load_index_from_storage(
#           StorageContext.from_defaults(persist_dir=save_dir),
#           service_context=sentence_context,
#       )

  return index

In [ ]:
# get the vector index
vector_index = get_build_index(
    documents=documents,
    llm=llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    sentence_window_size=3,
    save_dir="./vector_index/index",
    )

In [ ]:
from llama_index.core.postprocessor import MetadataReplacementPostProcessor, SentenceTransformerRerank

def get_query_engine(sentence_index, similarity_top_k=6, rerank_top_n=2):
  postproc = MetadataReplacementPostProcessor(target_metadata_key="window")

  rerank = SentenceTransformerRerank(
      top_n=rerank_top_n,
      model="BAAI/bge-reranker-base",
  )
  engine = sentence_index.as_query_engine(
        similarity_top_k = similarity_top_k,
        node_postprocessors = [postproc, rerank],
  )

  return engine

In [ ]:
query_engine = get_query_engine(
    sentence_index = vector_index,
    similarity_top_k = 6,
    rerank_top_n = 2,
    )

In [ ]:
from llama_index.core.response.notebook_utils import display_response

In [ ]:
query=input()
response = query_engine.query(query)
display_response(response)

name a water management digital technology


llama_tokenize_internal: Added a BOS token to the prompt as specified by the model but the prompt also starts with a BOS token. So now the final prompt starts with 2 BOS tokens. Are you sure this is what you want?
Llama.generate: prefix-match hit

llama_print_timings:        load time =    1411.60 ms
llama_print_timings:      sample time =      33.19 ms /    61 runs   (    0.54 ms per token,  1837.90 tokens per second)
llama_print_timings: prompt eval time =    1357.74 ms /   726 tokens (    1.87 ms per token,   534.71 tokens per second)
llama_print_timings:        eval time =    1647.48 ms /    60 runs   (   27.46 ms per token,    36.42 tokens per second)
llama_print_timings:       total time =    3069.27 ms /   786 tokens


**`Final Response:`** One digital technology for water management is 3D TRASAR™ Technology. It is used to optimize cooling water use by detecting and correcting variations in source water quality, reducing the demand for freshwater. This technology led to significant water savings for a steel maker mentioned in the context.